In [12]:
from soynlp.tokenizer import MaxScoreTokenizer, LTokenizer, RegexTokenizer
from pprint import pprint

## 1. MaxScoreTokenizer

띄어쓰기가 잘 안 된 한국어 문장을 토크나이징 해주는 unsupervised tokenizer이다. 점수를 잘 정의하면 단어 추출의 질이 상승한다

### [1-1] Score 값 설정

In [ ]:
scores = {'파스':0.3, '파스타':0.7, '좋아요' :0.2, '좋아':0.5}
#key : string형태, value: float 형태

### [1-2] MaxScoreTokenizer설정 및 문장 입력

In [13]:
tokenizer = MaxScoreTokenizer(scores = scores)
tokenizer.tokenize('파스타가좋아요')

['파스타', '가', '좋아', '요']

### [1-3] 출력

MaxTokenizer 출력 : (subword, begin, end, score)

In [16]:
print('flatten=False')
pprint(tokenizer.tokenize('난파스타가 좋아요', flatten =False))

print('\nflatten=True')
pprint(tokenizer.tokenize('난파스타가 좋아요', flatten= True))

flatten=False
[[('난', 0, 1, 0.0, 1), ('파스타', 1, 4, 0.7, 3), ('가', 4, 5, 0.0, 1)],
 [('좋아', 0, 2, 0.5, 2), ('요', 2, 3, 0.0, 1)]]

flatten=True
['난', '파스타', '가', '좋아', '요']


### [1-3] 한계점

    운동에 관한 기사라고 가정하면 '파스'에 좀 더 높은 점수를 줄 것이다. 
    그러나, 파스와, 파스타 둘 다 뜻하는 문장이 있다면 다시 고민해봐야 한다.
    문서의 길이가 늘어날 수록 score 선정이 복잡해 질 것이다

## 2. LTokenizer

띄어쓰기가 잘 돼있는 한국어 문서에는 MaxScoreTokenizer가 필요없다. 한국어는 L+[R] 구조이기 때문이다. 이때는 한 어절의 왼쪽에서 부터 글자 점수가 가장 높은 부분을 기준으로 토크나이징한다

### [2-1] Score 값 설정

In [17]:
from soynlp.tokenizer import LTokenizer 

scores = {'데이':0.5, '데이터':0.5, '데이터마이닝':0.5, '공부':0.5, '공부중':0.45}

### [2-2] LTokenizer 설정 및 문장 입력

    Cohesion score 를 이용하여 어절의 왼쪽에 위치한 substrings 중 가장 단어스러운 부분을 찾는 방식으로 간단한 비지도학습 기반 토크나이저를 만들 수 있습니다.

    뉴스처럼 띄어쓰기가 잘 되어 있는 한국어 텍스트의 어절 구조는 명확히 L + [R] 입니다. 어절의 왼쪽에 위치한 substring 중 단어 점수가 가장 높은 부분을 선택하면 어절을 L + [R] 로 나눌 수 있습니다.

In [21]:
tokenizer = LTokenizer(scores= scores)

In [28]:
print('\nflatten=True\nsent = 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터마이닝을 공부한다'))

print('\nflatten=False\nsent =데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터마이닝을 공부한다', flatten=False))

scores = {'데이':0.5, '데이터':0.5, '데이터마이닝':0.5, '공부':0.5, '공부중':0.45}


flatten=True
sent = 데이터마이닝을 공부한다
['데이터마이닝', '을', '공부', '한다']

flatten=False
sent =데이터마이닝을 공부한다
[('데이터마이닝', '을'), ('공부', '한다')]


In [29]:
print('\nflatten=True\nsent =데이터분석을 위해서 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터분석을 위해서 데이터마이닝을 공부한다'))

print('\nflatten=False\nsent =데이터분석을 위해서 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터분석을 위해서 데이터마이닝을 공부한다', flatten=False))

scores = {'데이':0.5, '데이터':0.5, '데이터마이닝':0.5, '공부':0.5, '공부중':0.45}


flatten=True
sent =데이터분석을 위해서 데이터마이닝을 공부한다
['데이터', '분석을', '위해서', '데이터마이닝', '을', '공부', '한다']

flatten=False
sent =데이터분석을 위해서 데이터마이닝을 공부한다
[('데이터', '분석을'), ('위해서', ''), ('데이터마이닝', '을'), ('공부', '한다')]


### [2-3] Tolerance

    한 어절에서 subword들의 점수의 차이가 그 어절의 점수 최대값과 tolerance 이하로 나타난다면, 길이가 가장 긴 어절을 선택한다
    --다시 이해해야함. 
    --어절의 점수 최대값 : 공부(0.5), tolerance : 0.1, 공부중(0.45)
    --tolerance + 공부중(0.45) > 공부(0.5) 때문에 출력되는게 아닐까?
    --tolerance 0.04로 설정해서 검증 : 맞네!
    검증 결과 : ('어절의 점수 최대값') < ('어절에서 두번째로 높은 값' + 'tolerance') 라면, 두번째로 높은 값이 출력됨

In [33]:
print('tolerance=0.0\nsent = 데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다'))

print('tolerance=0.1\nsent = 데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다', tolerance=0.1))

print('tolerance=0.04\nsent = 데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다', tolerance=0.04))

scores = {'데이':0.5, '데이터':0.5, '데이터마이닝':0.5, '공부':0.5, '공부중':0.45}

tolerance=0.0
sent = 데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부', '중이다']
tolerance=0.1
sent = 데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부중', '이다']
tolerance=0.04
sent = 데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부', '중이다']


## 3. RegexTokenizer

    RegexTokenizer는 regular expression을 이용해 언어가 달라지는순간들을 띄어쓴다. 
    영어의 경우에는 움라우트가 들어가는 경우가 있어서 알파벳 뿐만 아니라 라틴까지 포함한다. 
    또한, 숫자, 한글, 외래어(영어)등도 구분해준다.
    
    
    re.colpile('[가=힣]+') : 초/중/종성이 완전한 한국어의 시작부터 끝까지라는 의미
    re.compile('[ㄱ-ㅎ]+') : ㄱ부터 ㅎ까지 자음의 범위를 나타냄


    한국어에서 자음/모음이 단어 중간에 단어의 경계를 구분해주는 역할을 한다. 

In [36]:
tokenizer = RegexTokenizer()

sents =[
    '이렇게연속된문장은잘리지않습니다만', 
    '숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠',
    '띄어쓰기가 포함되어 있으면 이정보는10점!꼭띄워야죠',
    '아이고ㅋㅋ진짜?',
    '아이고ㅋㅋㅜㅜ진짜?'
]

for sent in sents:
    print(' %s\n->%s\n' % (sent, tokenizer.tokenize(sent)))

 이렇게연속된문장은잘리지않습니다만
->['이렇게연속된문장은잘리지않습니다만']

 숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠
->['숫자', '123', '이영어', 'abc', '에섞여있으면', 'ㅋㅋ', '잘리겠죠']

 띄어쓰기가 포함되어 있으면 이정보는10점!꼭띄워야죠
->['띄어쓰기가', '포함되어', '있으면', '이정보는', '10', '점', '!', '꼭띄워야죠']

 아이고ㅋㅋ진짜?
->['아이고', 'ㅋㅋ', '진짜', '?']

 아이고ㅋㅋㅜㅜ진짜?
->['아이고', 'ㅋㅋ', 'ㅜㅜ', '진짜', '?']



flatten = False 띄어쓰기 기준으로 토근을 나눠서 출력

In [35]:
tokenizer.tokenize('띄어쓰기가 포함되어 있으면 이정보는10점!꼭띄워야죠', flatten=False)

[['띄어쓰기가'], ['포함되어'], ['있으면'], ['이정보는', '10', '점', '!', '꼭띄워야죠']]